In [2]:
'''
Imports for testing purposes
'''

import requests
from bs4 import BeautifulSoup
import urllib
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import os
from ast import literal_eval
from datetime import datetime
from utils.base_templates import NewsArticle, ArticleCollection, Company, Insider

import marketscreener.scrapers.company_scraper as cs
import marketscreener.scrapers.news_scraper as ns
import marketscreener.scrapers.insider_scraper as ins
import marketscreener.scrapers.scraping_utils as su
import marketscreener.mongodb_utils as du

In [31]:
company_df = pd.read_csv('marketscreener/data/companies_info_2025-01-09.csv')
insider_df = pd.read_csv('marketscreener/data/full_insiders_info_2025-01-07.csv')

import re

def convert_net_worth_to_int(net_worth_str):
    # Use regex to find the number and the optional 'M' for millions
    try:
        match = re.search(r'(\d[\d\s]*)\s*M?\s*\$', str(net_worth_str))
        if match:
            # Extract the number part and remove spaces
            number_str = match.group(1).replace(' ', '')
            number = int(number_str)
            # Check if 'M' is present to multiply by 1,000,000
            if 'M' in net_worth_str:
                number *= 1_000_000
            elif 'B' in net_worth_str:
                number *= 1_000_000_000 
            return number
        return None
    except:
        return None

insider_df['net_worth_int'] = insider_df['net_worth'].apply(lambda x: convert_net_worth_to_int(x))

# Randomly sample 20 rows from each DataFrame
sampled_company_df = company_df.sample(n=20, random_state=1)
sampled_insider_df = insider_df.sample(n=20, random_state=42)

# Save the sampled DataFrames to separate CSV files
sampled_company_df.to_csv('sampled_companies.csv', index=False)
sampled_insider_df.to_csv('sampled_insiders.csv', index=False)



In [3]:
def find_company_and_insiders_from_article(article_link: str):
    # Step 1: Fetch the article from the Articles collection
    articles_collection = du.get_db_collection(du.DB_NAME, 'Articles')
    article = articles_collection.find_one({"link": article_link})
    print(article)
    if not article:
        article = ns.get_article_from_link(article_link)
        if not article:
            return None
    # Step 2: Fetch the company linked to this article
    company_link = 'https://www.marketscreener.com' + article.get('company_link', '')
    companies_collection = du.get_db_collection(du.DB_NAME, 'Companies')
    company = companies_collection.find_one({"link": company_link})
    
    if not company:
        # If the company is not found, scrape it
        company_name, country, ticker, isin, industry, sector, company_profile, executives = cs.extract_company_info(company_link, full=True)
        if not ticker:
            print("Company not found and could not be scraped.")
            return None 
        company = { # Find industry, sector, country
            'name': company_name,
            'link': company_link,
            'ticker': ticker,
            'isin': isin,
            'industry': industry,
            'sector': sector,
            'country': country,
            'profile': company_profile,
            'executives': executives
        }
        # Save the scraped company data to the database
        companies_collection.insert_one(company)

    # Step 3: Get executive 'href' links to query the Insiders collection
    executives = su.str_to_dict_expansion(company.get('executives', '{}'))
    insider_links = {
        name: info['href']  # Use the name as the key and extract 'href' from the info dictionary
        for exec_category in executives.values() 
        for name, info in exec_category.items()  # Iterate over items to get both name and info
    }
    insider_links_list = list(insider_links.values())

    # Step 4: Fetch insiders based on the collected links
    insiders_collection = du.get_db_collection(du.DB_NAME, 'Insiders')
    insiders = list(insiders_collection.find({"link": {"$in": insider_links_list}}))

    if not insiders:
        # If insiders are not found, scrape them
        for name, link in insider_links.items():
            # Pass the name as an additional argument
            # print(name, link)
            insider = ins.extract_insider_info(name, link)
            if insider:
                insiders.append(insider)
                # Save the scraped insider data to the database
                insiders_collection.insert_one(insider)

    # Return the collected data as a dictionary
    return {
        "article": article,
        "company": company,
        "insiders": insiders
    }

In [5]:
def generate_markdown_report(article_link: str, output_dir: str = "reports"):
    # Fetch article, company, and insiders data
    data = find_company_and_insiders_from_article(article_link)
    if data is None:
        return

    article = data["article"]
    company = data["company"]
    insiders = data["insiders"]

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Generate the markdown file name using the article headline and date
    headline = article.get('headline', 'unknown_headline').replace(' ', '_')
    publication_date = article.get('publication_date', 'unknown_date')
    file_name = f"{headline}_{publication_date}.md"
    file_path = os.path.join(output_dir, file_name)

    # Write the markdown report
    with open(file_path, 'w') as md_file:
        md_file.write(f"# {article.get('headline')}\n")
        md_file.write(f"**Publication Date:** {article.get('publication_date')}\n")
        md_file.write(f"**Category:** {article.get('category')}\n")
        md_file.write(f"**Source:** {article.get('source')}\n\n")
        md_file.write("## Article Content\n")
        md_file.write(f"{article.get('content')}\n\n")
        md_file.write("## Company Information\n")
        md_file.write(f"- **Name:** {company.get('name')}\n")
        md_file.write(f"- **ISIN:** {company.get('isin')}\n")
        md_file.write(f"- **Ticker:** {company.get('ticker')}\n")
        md_file.write(f"- **Industry:** {company.get('industry')}\n")
        md_file.write(f"- **Sector:** {company.get('sector')}\n")
        md_file.write(f"- **Country:** {company.get('country')}\n")
        profile = company.get('profile', '').replace('-', '\\-')
        md_file.write(f"- **Profile:**\n {profile}\n\n")
        md_file.write("## Linked Insiders\n")
        for insider in insiders:
            md_file.write(f"- **Name:** {insider.get('name')}\n")
            md_file.write(f"- **Current Position:** {insider.get('current_position')}\n")
            md_file.write(f"- **Current Company:** {insider.get('current_company')}\n")
            md_file.write(f"- **Company URL:** {insider.get('company_url')}\n")
            md_file.write(f"- **Net Worth:** {insider.get('net_worth', 'N/A')}\n")
            known_holdings = su.str_to_dict_expansion(insider.get('known_holdings', '{}'))
            md_file.write(f"- **Known Holdings:**\n")
            for company, details in known_holdings.items():
                md_file.write(f"  - **{company}:**\n")
                md_file.write(f"    - **Link:** {details.get('link', 'N/A')}\n")
                md_file.write(f"    - **Date:** {details.get('date', 'N/A')}\n")
                md_file.write(f"    - **Number of Shares:** {details.get('number_of_shares', 'N/A')}\n")
                valuation_str = details.get('valuation', 'N/A').replace('\n', '')
                md_file.write(f"    - **Valuation:** {valuation_str}\n")
                md_file.write(f"    - **Valuation Date:** {details.get('valuation_date', 'N/A')}\n")
            md_file.write(f"- **Age:** {insider.get('age', 'N/A')}\n")
            md_file.write(f"- **Industries:** {', '.join(su.str_to_dict_expansion(insider.get('industries', '[]')))}\n")
            md_file.write(f"- **Summary:** {insider.get('summary', 'N/A')}\n")
            md_file.write("- **Active Positions:**\n")
            active_positions = su.str_to_dict_expansion(insider.get('active_positions', '{}'))
            for position, date in active_positions.items():
                md_file.write(f"    - {position}: {date}\n")
            md_file.write("- **Former Positions:**\n")
            former_positions = su.str_to_dict_expansion(insider.get('former_positions', '{}'))
            for position, date in former_positions.items():
                md_file.write(f"    - {position}: {date}\n")
            md_file.write("- **Education:**\n")
            trainings = su.str_to_dict_expansion(insider.get('trainings', '{}'))
            for training, details in trainings.items():
                md_file.write(f"    - {training}: {details}\n")
            md_file.write("\n---\n\n")

    print(f"Markdown report saved to {file_path}")

In [ ]:
current_date = datetime.now().strftime('%Y-%m-%d')
news_df = pd.read_csv(f'marketscreener/data/marketscreener_articles_{current_date}.csv')
news_df.head()

In [7]:
'''
Going from insider table and company table to insider-company table
'''

company_df = pd.read_csv('marketscreener/data/companies_info_2025-01-09.csv')
insider_df = pd.read_csv('marketscreener/data/full_insiders_info_2025-01-07.csv')

display(company_df.head())
display(insider_df.head())


,name,isin,ticker,industry,sector,country,executives,profile,link
0,ARAMCO,SA14TG012N13,2222,Energy - Fossil Fuels,Integrated Oil & Gas,SAUDI ARABIA,{'Manager': {'Amin Al-Nasser': {'href': '/insi...,"Saudi Arabian Oil Company, also known as Saudi...",https://www.marketscreener.com/quote/stock/ARA...
1,EXXON MOBIL CORPORATION,US30231G1022,XOM,Energy - Fossil Fuels,Integrated Oil & Gas,UNITED STATES,{'Manager': {'Darren Woods': {'href': '/inside...,Exxon Mobil Corporation is an oil group organi...,https://www.marketscreener.com/quote/stock/EXX...
2,CHEVRON CORPORATION,US1667641005,CVX,Energy - Fossil Fuels,Integrated Oil & Gas,UNITED STATES,{'Manager': {'Michael Wirth': {'href': '/insid...,Chevron Corporation is one of the world leadin...,https://www.marketscreener.com/quote/stock/CHE...
3,PETROCHINA COMPANY LIMITED,CNE1000003W8,857,Energy - Fossil Fuels,Integrated Oil & Gas,CHINA,{'Manager': {'Hua Wang': {'href': '/insider/HU...,PetroChina Co Ltd is a China-based company pri...,https://www.marketscreener.com/quote/stock/PET...
4,SHELL PLC,GB00BP6MXD84,SHEL,Energy - Fossil Fuels,Integrated Oil & Gas,UNITED KINGDOM,{'Manager': {'Wael Sawan': {'href': '/insider/...,Shell plc specializes in oil and natural gas p...,https://www.marketscreener.com/quote/stock/SHE...


,name,current_position,current_company,company_url,net_worth,known_holdings,age,industries,summary,active_positions,former_positions,trainings,link
0,Barry Diller,Chairman at IAC INC.,IAC INC.,/quote/stock/IAC-INC-109581170/,Net worth: 1 457 M\n $ as of 2024-10-30,"{'EXPEDIA GROUP, INC.': {'link': '/quote/stock...",82,"['Internet Software/Services', 'Miscellaneous'...",Barry Charles Diller is an American businesspe...,"{'EXPEDIA GROUP, INC.': 'Chairman', 'IAC INC.'...",{},{},/insider/BARRY-DILLER-A0003W/
1,Larry Ellison,Founder at ORACLE CORPORATION,ORACLE CORPORATION,/quote/stock/ORACLE-CORPORATION-13620698/,Net worth: 192 B\n $ as of 2024-10-30,{'ORACLE CORPORATION': {'link': '/quote/stock/...,80,"['Internet Software/Services', 'Commercial Ser...",Lawrence Joseph Ellison is an entrepreneur who...,"{'ORACLE CORPORATION': 'Founder', 'CERNER CORP...","{'TESLA, INC.': 'Director/Board Member', 'Rete...",{},/insider/LARRY-ELLISON-A0004C/
2,Kirk Kerkorian,NaN,NaN,NaN,NaN,{},106,"['Investment Managers', 'Consumer Services', '...","Kerkor 'Kirk' Kerkorian (born June 6, 1917), k...",{},"{'MGM RESORTS INTERNATIONAL': 'Founder', 'Trac...",{},/insider/KIRK-KERKORIAN-A00099/
3,George Soros,NaN,NaN,NaN,NaN,{},94,"['Commercial Services', 'Financial Conglomerat...",George Soros lived the Hungary’s Nazi occupati...,"{'Soros Fund Management LLC': 'Founder', 'Open...","{'Soros Economic Development Fund': 'Founder',...",{'London School of Economics & Political Scien...,/insider/GEORGE-SOROS-A000HG/
4,Bill Gates,NaN,NaN,NaN,Net worth: 42 328 M\n $ as of 2024-10-30,{'MICROSOFT CORPORATION': {'link': '/quote/sto...,68,"['Telecommunications Equipment', 'Commercial S...","William Henry ""Bill"" Gates III (born 28 Octobe...",{'Bill & Melinda Gates Foundation Trust': 'Cha...,{'MICROSOFT CORPORATION': 'Chief Executive Off...,{},/insider/BILL-GATES-A000O8/


In [9]:
from nameparser import HumanName
from nameparser.config import CONSTANTS
CONSTANTS.capitalize_name = True

def split_name(x):
    name = HumanName(x)
    return name.first, name.middle, name.last

insider_df[['First name', 'Middle name', 'Last name']] = insider_df['name'].apply(lambda x: pd.Series(split_name(x)))


In [13]:
insider_df.describe()

,age
count,29144.000000
mean,59.623250
std,9.861532
min,24.000000
25%,53.000000
50%,59.000000
75%,66.000000
max,106.000000


In [16]:
import re

def convert_net_worth_to_int(net_worth_str):
    # Use regex to find the number and the optional 'M' for millions
    try:
        match = re.search(r'(\d[\d\s]*)\s*M?\s*\$', str(net_worth_str))
        if match:
            # Extract the number part and remove spaces
            number_str = match.group(1).replace(' ', '')
            number = int(number_str)
            # Check if 'M' is present to multiply by 1,000,000
            if 'M' in net_worth_str:
                number *= 1_000_000
            elif 'B' in net_worth_str:
                number *= 1_000_000_000 
            return number
        return None
    except:
        return None

insider_df['net_worth_int'] = insider_df['net_worth'].apply(lambda x: convert_net_worth_to_int(x))
insider_df.head()

In [18]:
insider_df['net_worth_int'] = insider_df['net_worth'].apply(lambda x: convert_net_worth_to_int(x))
insider_df.head()

,name,current_position,current_company,company_url,net_worth,known_holdings,age,industries,summary,active_positions,former_positions,trainings,link,First name,Middle name,Last name,net_worth_int
0,Lamberto Andreotti,"Director/Board Member at CORTEVA, INC.","CORTEVA, INC.",/quote/stock/CORTEVA-INC-59241389/,2 M $ as of 2024-09-29,{'CORTEVA INC': {'link': '/quote/stock/CORTEVA...,74.0,"['Investment Managers', 'Consumer Non-Durables...",Lamberto Andreotti is Member of Pharmaceutical...,{'E I DU PONT DE NEMOURS AND CO': 'Director/Bo...,"{'Essex Woodlands Management, Inc.': 'Consulta...",{'Massachusetts Institute of Technology': 'Gra...,/insider/LAMBERTO-ANDREOTTI-A0104Q/,Lamberto,,Andreotti,2000000.0
1,Sam Allen,Director/Board Member at WHIRLPOOL CORPORATION,WHIRLPOOL CORPORATION,/quote/stock/WHIRLPOOL-CORPORATION-14872/,3 M $ as of 2024-09-29,{'WHIRLPOOL CORPORATION': {'link': '/quote/sto...,70.0,"['Investment Managers', 'Miscellaneous Commerc...",Samuel R. Allen is a businessperson who has be...,{'WHIRLPOOL CORPORATION': 'Director/Board Memb...,"{'DEERE & COMPANY': 'Chief Executive Officer',...",{'Purdue University': 'Undergraduate Degree'},/insider/SAM-ALLEN-A013Y3/,Sam,,Allen,3000000.0
2,Tommy Andrews,NaN,NaN,NaN,NaN,{},73.0,"['Data Processing Services', 'Technology Servi...",Tommy L. Andrews has been Executive Vice Presi...,{},"{'CORPAY, INC.': 'Chief Operating Officer', 'W...",{},/insider/TOMMY-ANDREWS-A01840/,Tommy,,Andrews,NaN
3,Martin Anstice,NaN,NaN,NaN,NaN,{},57.0,"['Miscellaneous Commercial Services', 'Produce...",Martin Brian Anstice is currently a Director a...,{'The Chartered Institute of Management Accoun...,{'LAM RESEARCH CORPORATION': 'Chief Executive ...,{},/insider/MARTIN-ANSTICE-A01CDF/,Martin,,Anstice,NaN
4,Mark McLaughlin,"Chairman at QUALCOMM, INC.","QUALCOMM, INC.",/quote/stock/QUALCOMM-INC-4897/,90 M $ as of 2024-09-29,"{'PALO ALTO NETWORKS, INC.': {'link': '/quote/...",58.0,"['Internet Software/Services', 'Investment Man...",Mark D. McLaughlin holds the position of Chair...,"{'QUALCOMM, INC.': 'Chairman', 'SNOWFLAKE INC....","{'PALO ALTO NETWORKS, INC.': 'Chief Executive ...",{'US Military Academy': 'Undergraduate Degree'...,/insider/MARK-MCLAUGHLIN-A01HGW/,Mark,,McLaughlin,90000000.0


In [22]:
insider_df = insider_df.sort_values(by='net_worth_int', ascending=False)
insider_df.to_csv('processed_insider_df.csv', index=False)


In [9]:
import json
from ast import literal_eval

hk_company_df = company_df[company_df['country'] == "HONG KONG"]

# Extract the 'executives' column and convert it to a dictionary
executives_dict = {}

for index, row in hk_company_df.iterrows():
    company_name = row['name']
    executives = literal_eval(row['executives'])
    
    # Iterate over each executive category and their details
    for category, execs in executives.items():
        for exec_name, details in execs.items():
            href = details.get('href')
            if href:
                # Add to the dictionary
                if company_name not in executives_dict:
                    executives_dict[company_name] = {}
                executives_dict[company_name][exec_name] = href

# Save the dictionary to a JSON file
with open('hk_executives_hrefs.json', 'w') as json_file:
    json.dump(executives_dict, json_file, indent=4)

print("Executives hrefs saved to 'executives_hrefs.json'")

# Load the executives hrefs dictionary
with open('hk_executives_hrefs.json', 'r') as json_file:
    executives_dict = json.load(json_file)

# Extract all hrefs from the executives_dict
all_hrefs = set()
for company, executives in executives_dict.items():
    for exec_name, href in executives.items():
        all_hrefs.add(href)

# Filter the insider_df to only include rows where the 'link' is in all_hrefs
filtered_insider_df = insider_df[insider_df['link'].apply(lambda x: x.split('https://www.marketscreener.com/')[-1]).isin(all_hrefs)]

# Display the filtered DataFrame
print(filtered_insider_df)

# Optionally, save the filtered DataFrame to a new CSV file
filtered_insider_df.to_csv('filtered_hk_insiders_info_0109.csv', index=False)

Executives hrefs saved to 'executives_hrefs.json'
                     name                                   current_position  \
77           Shau Kee Lee  Founder at HENDERSON LAND DEVELOPMENT COMPANY ...   
709           David Banks                                                NaN   
858       Kuo Fung Ch'ien  Director/Board Member at CHINA RESOURCES POWER...   
872        Jian Jiang Cai                                                NaN   
931       Chung Kong Chow         Director/Board Member at AIA GROUP LIMITED   
...                   ...                                                ...   
83593              Yao Li  Chief Administrative Officer at CHINA MERCHANT...   
83999            Wang Xin           Director of Finance/CFO at CNOOC LIMITED   
84456          Mick Myers          Chief Executive Officer at NUCHEV LIMITED   
84825        Camille Jojo  Compliance Officer at TECHTRONIC INDUSTRIES CO...   
85212  Michael Fitzgerald  Director of Finance/CFO at MTR CORPORATION 

In [7]:
# Assuming company_df is correct and contains all necessary information
# Load the data
company_df = pd.read_csv('marketscreener/data/companies_info_2025-01-09.csv')
# company columns are: 'name', 'isin', 'ticker', 'industry', 'sector', 'profile', 'link', 'country'
insider_df = pd.read_csv('marketscreener/data/full_insiders_info_2025-01-07.csv') 
# insider columns are: 'name', 'current_position', 'current_company', 'company_url', 'net_worth', 'known_holdings', 'age', 'industries', 'summary', 'active_positions', 'former_positions', 'trainings', 'link'

# Clean the company links
company_df['link'] = company_df['link'].str.replace('https://www.marketscreener.com', '')

# Extract executives
company_df['executives'] = company_df['executives'].apply(su.str_to_dict_expansion)

# Create a list to store the insider-company relationships
insider_company_relationships = []

# Iterate over each company
for _, company in company_df.iterrows():
    company_name = company['name']
    company_link = company['link']
    executives = company['executives']
    
    # Iterate over each executive
    for exec_category in executives.values():
        for name, info in exec_category.items():
            insider_link = info['href']
            
            # Directly create the relationship assuming the data is correct
            insider_company_relationships.append({
                'insider_name': name,
                'company_name': company_name,
                'insider_link': insider_link,
                'company_link': company_link
            })

# Convert the relationships to a DataFrame
insider_company_df = pd.DataFrame(insider_company_relationships)
# Drop duplicate insider-company pairs
insider_company_df = insider_company_df.drop_duplicates(subset=['insider_link', 'company_link'])

display(insider_company_df)

# Save to a CSV or any other format as needed
insider_company_df.to_csv('insider_company_relationships.csv', index=False)

,insider_name,company_name,insider_link,company_link
0,Amin Al-Nasser,ARAMCO,/insider/AMIN-AL-NASSER-A11V40/,/quote/stock/ARAMCO-103505448/
1,Ziad Al-Murshed,ARAMCO,/insider/ZIAD-AL-MURSHED-A2PTVD/,/quote/stock/ARAMCO-103505448/
2,Wail Al Jaafari,ARAMCO,/insider/WAIL-AL-JAAFARI-A3VAUM/,/quote/stock/ARAMCO-103505448/
3,Ahmad Al-Khowaiter,ARAMCO,/insider/AHMAD-AL-KHOWAITER-A1BAPY/,/quote/stock/ARAMCO-103505448/
4,Nabeel Al-Jama,ARAMCO,/insider/NABEEL-AL-JAMA-A33D8Y/,/quote/stock/ARAMCO-103505448/
...,...,...,...,...
317856,Torbjörn Sannerstedt,ACADEMEDIA AB,/insider/TORBJORN-SANNERSTEDT-A103YU/,/quote/stock/ACADEMEDIA-AB-30049783/
317857,Per Gösta Franzén,ACADEMEDIA AB,/insider/PER-GOSTA-FRANZEN-A1OS24/,/quote/stock/ACADEMEDIA-AB-30049783/
317858,Royne Berglund,ACADEMEDIA AB,/insider/ROYNE-BERGLUND-A1NLMW/,/quote/stock/ACADEMEDIA-AB-30049783/
317859,Carl Fredrik Sven Grevelius,ACADEMEDIA AB,/insider/CARL-FREDRIK-SVEN-GREVELIUS-A07551/,/quote/stock/ACADEMEDIA-AB-30049783/


In [8]:
from neo4j import GraphDatabase
import pandas as pd
from tqdm import tqdm
import os

# Load your data
company_df = pd.read_csv('marketscreener/data/companies_info_2024-11-01.csv')
insider_df = pd.read_csv('marketscreener/data/full_insiders_info_2024-10-22.csv')
insider_company_df = pd.read_csv('insider_company_relationships.csv')

# Calculate memory usage for each DataFrame
company_memory = company_df.memory_usage(deep=True).sum()
insider_memory = insider_df.memory_usage(deep=True).sum()
insider_company_memory = insider_company_df.memory_usage(deep=True).sum()

# Convert bytes to megabytes for easier reading
company_memory_mb = company_memory / (1024 ** 2)
insider_memory_mb = insider_memory / (1024 ** 2)
insider_company_memory_mb = insider_company_memory / (1024 ** 2)

print(f"Company DataFrame memory usage: {company_memory_mb:.2f} MB")
print(f"Insider DataFrame memory usage: {insider_memory_mb:.2f} MB")
print(f"Insider-Company DataFrame memory usage: {insider_company_memory_mb:.2f} MB")

Company DataFrame memory usage: 57.06 MB
Insider DataFrame memory usage: 66.88 MB
Insider-Company DataFrame memory usage: 100.69 MB


In [ ]:
# Calculate memory usage for each column in the Company DataFrame
company_column_memory = company_df.memory_usage(deep=True)

# Convert bytes to megabytes for easier reading
company_column_memory_mb = company_column_memory / (1024 ** 2)

print("Company DataFrame column memory usage (in MB):")
print(company_column_memory_mb)

# Calculate memory usage for each column in the Insider DataFrame
insider_column_memory = insider_df.memory_usage(deep=True)

# Convert bytes to megabytes for easier reading
insider_column_memory_mb = insider_column_memory / (1024 ** 2)

print("\nInsider DataFrame column memory usage (in MB):")
print(insider_column_memory_mb)

Company DataFrame column memory usage (in MB):
Index          0.000126
name           0.599184
isin           0.492079
ticker         0.440112
industry       0.538600
sector         0.601177
profile        5.723660
executives    47.264978
link           0.929356
country        0.468667
dtype: float64

Insider DataFrame column memory usage (in MB):
Index                0.000126
name                 2.570023
current_position     2.959524
current_company      2.360361
company_url          2.809028
net_worth            1.790947
known_holdings       6.057665
age                  0.285233
industries           7.115273
summary             19.536579
active_positions     8.166383
former_positions     5.849279
trainings            4.244407
link                 3.136507
dtype: float64


In [ ]:
from neo4j import GraphDatabase
import pandas as pd
from tqdm import tqdm
import os
from multiprocessing import Pool, cpu_count

# Load your data
insider_company_df = pd.read_csv('insider_company_relationships.csv').head(30000)

# Connect to Neo4j
uri = "neo4j+s://79f59118.databases.neo4j.io:7687"  # Update with your Neo4j URI
username = "neo4j"  # Update with your Neo4j username
password = os.getenv("NEO4J_KEY")  # Update with your Neo4j password

driver = GraphDatabase.driver(uri, auth=(username, password))

def reset_graph(tx):
    tx.run("MATCH (n) DETACH DELETE n")
    print('Graph Refreshed')

def create_nodes_and_relationships(rows):
    with driver.session() as session:
        for i, row in tqdm(rows.iterrows(), total=rows.shape[0], desc="Creating nodes and relationships"):
            session.execute_write(lambda tx: tx.run("""
                MERGE (i:Insider {link: $insider_link})
                SET i.name = $insider_name
                MERGE (c:Company {link: $company_link})
                SET c.name = $company_name
                MERGE (i)-[:INSIDER_AT]-(c)
            """, insider_link=row['insider_link'], insider_name=row['insider_name'],
                 company_link=row['company_link'], company_name=row['company_name']))

def process_rows(start_index, end_index):
    rows = insider_company_df.iloc[start_index:end_index]
    create_nodes_and_relationships(rows)

if __name__ == '__main__':
    with driver.session() as session:
        # Reset the graph
        session.execute_write(reset_graph)

    # Define the checkpoint interval
    checkpoint_interval = 3000

    # Use multiprocessing to parallelize the creation of nodes and relationships
    with Pool(processes=cpu_count()) as pool:
        for start_index in range(0, len(insider_company_df), checkpoint_interval):
            end_index = min(start_index + checkpoint_interval, len(insider_company_df))
            pool.apply_async(process_rows, (start_index, end_index))
            print(f'Processing index {start_index}')

        pool.close()
        pool.join()

    driver.close()

In [ ]:
# Load your data
company_df = pd.read_csv('marketscreener/data/companies_info_2025-01-09.csv')
insider_df = pd.read_csv('marketscreener/data/full_insiders_info_2025-01-07.csv')

# Drop the 'executives' column from the company DataFrame
company_df = company_df.drop(columns=['executives'])

# Drop the 'summary' column from the insider DataFrame
insider_df = insider_df.drop(columns=['summary'])

driver = GraphDatabase.driver(uri, auth=(username, password))

def create_nodes_and_relationships(tx, df, label, unique_key, name_field):
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Creating {label} nodes"):
        properties = {k: v for k, v in row.items() if k not in [unique_key, name_field, 'executives']}
        tx.run(f"""
            MERGE (n:{label} {{ {unique_key}: $unique_key }})
            SET n.name = $name
            SET n += $properties
        """, unique_key=row[unique_key], name=row[name_field], properties=properties)

def create_relationships(tx, df):
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Creating relationships"):
        tx.run("""
            MATCH (i:Insider {link: $insider_link})
            MATCH (c:Company {link: $company_link})
            MERGE (i)-[:INSIDER_AT]-(c)
        """, insider_link=row['insider_link'], company_link=row['company_link'])

with driver.session() as session:
    # Create Company nodes
    session.execute_write(create_nodes_and_relationships, company_df, "Company", "link", "name")
    
    # Create Insider nodes
    session.execute_write(create_nodes_and_relationships, insider_df, "Insider", "link", "name")
    
    # Create relationships
    session.execute_write(create_relationships, insider_company_df)

driver.close()

In [ ]:

# Load the data
insider_company_df = pd.read_csv('insider_company_relationships.csv')

# Check the number of distinct insiders
distinct_insiders = insider_company_df['insider_link'].nunique()
print(f"Number of distinct insiders: {distinct_insiders}")

# Check the number of distinct companies
distinct_companies = insider_company_df['company_link'].nunique()
print(f"Number of distinct companies: {distinct_companies}")

# Check for duplicate insider-company pairs
duplicate_pairs = insider_company_df.duplicated(subset=['insider_link', 'company_link']).sum()
print(f"Number of duplicate insider-company pairs: {duplicate_pairs}")

Number of distinct insiders: 261669
Number of distinct companies: 7325
Number of duplicate insider-company pairs: 0


In [ ]:
# Load the data
insider_company_df = pd.read_csv('insider_company_relationships.csv')

# Count the number of companies each insider is associated with
insider_company_counts = insider_company_df.groupby('insider_link')['company_link'].nunique()

# Calculate the frequency of each distinct count
count_frequencies = insider_company_counts.value_counts().sort_index()

# Plot the distribution using a bar chart
plt.figure(figsize=(10, 6))
plt.bar(count_frequencies.index, count_frequencies.values, color='b', alpha=0.7)

plt.title('Frequency of Distinct Company Counts per Insider')
plt.yscale('log')
plt.xlabel('Number of Companies')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)

# Set x-axis labels to every integer
plt.xticks(range(count_frequencies.index.min()-1, count_frequencies.index.max() + 1))

plt.show()

# Load the data
insider_company_df = pd.read_csv('insider_company_relationships.csv')

# Count the number of insiders each company is associated with
company_insider_counts = insider_company_df.groupby('company_link')['insider_link'].nunique()

# Calculate the frequency of each distinct count
count_frequencies = company_insider_counts.value_counts().sort_index()

# Plot the distribution using a bar chart
plt.figure(figsize=(10, 6))
plt.bar(count_frequencies.index, count_frequencies.values, color='b', alpha=0.7)

plt.title('Frequency of Distinct Insider Counts per Company')
plt.xlabel('Number of Insiders')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)

# Set x-axis labels to every integer
#plt.yscale('log')
plt.xticks(range(count_frequencies.index.min() - 1, count_frequencies.index.max(), 10))

plt.show()


In [ ]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

client = OpenAI(
  api_key=os.getenv("OPENAI_API_KEY")
)

classification_prompt = '''
You will be provided with a news article. Your task is to extract and identify relationships between companies, individuals, and their roles. 

For each relationship, provide:
1. The company name.
2. The individual's name.
3. The role of the individual within the company.
4. The portion of article used to explain why the individual is associated with the company in the specified role.

Format your response as a JSON object like this:

{
  "INDIVIDUAL_NAME": {
    "COMPANY_NAME_1": {
      "role": "role_description",
      "article_extract": "reason_for_association"
    },
    "COMPANY_NAME_2": {
      "role": "role_description",
      "article_extract": "reason_for_association"
    }
  }
}

Please ensure the JSON is well-structured and includes all relevant details.
'''
def classify_article(article_text: str, model: str = "gpt-4o", classification_prompt: str = classification_prompt) -> dict:
  try:
    if model == 'o1-preview':
      completion = client.chat.completions.create(
        model=model,
        messages=[
          {"role": "user", "content": classification_prompt},
          {"role": "user", "content": article_text}
        ]
      )
      candidate_json = completion.choices[0].message.content
      format_prompt = f"Format the following json-like content into json: {candidate_json}"
      completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
          {"role": "user", "content": format_prompt},
        ],
        response_format={"type": "json_object"}
      )
    else:
      completion = client.chat.completions.create(
        model=model, #"o1-preview",
        messages=[
          {"role": "user", "content": classification_prompt},
          {"role": "user", "content": article_text}
        ],
        response_format={"type": "json_object"}
      )

    return json.loads(completion.choices[0].message.content)
  except Exception as e:
    print(f"An error occurred: {e}")
    return {}

In [ ]:
'''
Test using archive.is to access paywalled news sites
'''

import requests
from bs4 import BeautifulSoup
import urllib.parse

def search_archive_is(original_url: str) -> str:
    """
    Searches archive.is for an existing archived URL of the given original URL.
    """
    search_url = f"https://archive.is/{original_url}"
    response = requests.get(search_url, headers=HEADERS)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        # The search results usually have <a> tags pointing to the archived URL
        result = soup.find_all('a', href=True)
        if result:
            return result
    return ''

url = 'https://www.reuters.com/business/healthcare-pharmaceuticals/bicara-therapeutics-targets-265-mln-proceeds-upsized-us-ipo-2024-09-11/'
test = search_archive_is(url)
for item in test:
    print(item['href'])

In [24]:

from typing import Dict
import requests
import random
from bs4 import BeautifulSoup
import pandas as pd

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/57.0",
    # Add more user agents as needed
]

def get_random_user_agent() -> Dict[str, str]:
    """
    Returns a random user agent from the predefined list.

    :return: A dictionary with a random user agent string.
    """
    return {'User-Agent': random.choice(USER_AGENTS)}

url_2019 = "https://www.hurun.net/en-us/info/detail?num=CE08472BB47D"
url_2020 = "https://www.hurun.net/en-us/info/detail?num=1E096ECED920"
url_2021 = ""

try:
    response = requests.get(url_2019, headers=get_random_user_agent())
    response.raise_for_status()
except requests.RequestException as e:
    print(f"Error: {e}")
    
soup = BeautifulSoup(response.content, "html.parser")


In [25]:
# Find all rows in the table
rows = soup.find_all('tr')

# Extract data
table_data = []
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    table_data.append([ele for ele in cols if ele])  # Get rid of empty values

# Print the extracted table data
flag = False
rows = []
for row in table_data:
    if flag:
        rows.append(row)
    if 'Rank change' in row:
        flag = True
        cols = row

# Create the initial DataFrame
df = pd.DataFrame(data=rows, columns=cols)

# Function to split names and ages while retaining other columns
def split_names_ages(row):
    try:
        if '&' in row['Name'] and 'family' not in row['Name']:
            names = " ".join(row['Name'].split()).split(' & ')
            ages = row['Age'].split(', ')
            if len(ages) < len(names):
                ages.append('-')
            # Create a DataFrame for each split name and age, retaining other columns
            return pd.DataFrame({
                'Name': names,
                'Age': ages,
                **{col: row[col] for col in row.index if col not in ['Name', 'Age']}
            })
        else:
            return pd.DataFrame({
                'Name': [row['Name'].replace('family', '').replace('&', ''.strip())],
                'Age': [row['Age']],
                **{col: [row[col]] for col in row.index if col not in ['Name', 'Age']}
            })
    except Exception as e:
        print(f"Error processing row: {row}, Error: {e}")
        return pd.DataFrame({
            'Name': [row['Name']],
            'Age': [row['Age']],
            **{col: [row[col]] for col in row.index if col not in ['Name', 'Age']}
        })

# Apply the function and explode the DataFrame
exploded_dfs = df.apply(split_names_ages, axis=1).tolist()

# Concatenate the exploded DataFrames
result_df = pd.concat(exploded_dfs, ignore_index=True)

# Display the result
print(result_df)

               Name Age  Rank Rank change Wealth US$m Change in wealth  \
0          Ma Yun    55     1           0      39,000               2%   
1        Ma Huateng  48     2           1      37,000               8%   
2         Xu Jiayin  61     3          -1      30,000             -16%   
3      He Xiangjian  77     4           2      26,000              38%   
4       He Jianfeng  52     4           2      26,000              38%   
...             ...  ..   ...         ...         ...              ...   
2167    Zheng Dabao  67  1727          48         290               0%   
2168  Zheng Youquan  65  1727          48         290               0%   
2169    Zheng Zhong  52  1727          48         290               0%   
2170         Qiu Ai   -  1727          48         290               0%   
2171    Zou Jianhan  50  1727        -257         290             -26%   

                   Business Place of   Residence      Key Source of   Wealth  
0                   Alibaba     

In [26]:
display(result_df.head())

insider_df = pd.read_csv('marketscreener/data/full_insiders_info_2025-01-07.csv')
insider_df.head()

,Name,Age,Rank,Rank change,Wealth US$m,Change in wealth,Business,Place of Residence,Key Source of Wealth
0,Ma Yun,55,1,0,"39,000",2%,Alibaba,Zhejiang,"E-commerce, fintech"
1,Ma Huateng,48,2,1,"37,000",8%,Tencent,Guangdong,Internet services
2,Xu Jiayin,61,3,-1,"30,000",-16%,Evergrande,Guangdong,"Real estate, investments"
3,He Xiangjian,77,4,2,"26,000",38%,Midea,Guangdong,Home appliances
4,He Jianfeng,52,4,2,"26,000",38%,Midea,Guangdong,Home appliances


,name,current_position,current_company,company_url,net_worth,known_holdings,age,industries,summary,active_positions,former_positions,trainings,link
0,Barry Diller,Chairman at IAC INC.,IAC INC.,/quote/stock/IAC-INC-109581170/,Net worth: 1 457 M\n $ as of 2024-10-30,"{'EXPEDIA GROUP, INC.': {'link': '/quote/stock...",82,"['Internet Software/Services', 'Miscellaneous'...",Barry Charles Diller is an American businesspe...,"{'EXPEDIA GROUP, INC.': 'Chairman', 'IAC INC.'...",{},{},/insider/BARRY-DILLER-A0003W/
1,Larry Ellison,Founder at ORACLE CORPORATION,ORACLE CORPORATION,/quote/stock/ORACLE-CORPORATION-13620698/,Net worth: 192 B\n $ as of 2024-10-30,{'ORACLE CORPORATION': {'link': '/quote/stock/...,80,"['Internet Software/Services', 'Commercial Ser...",Lawrence Joseph Ellison is an entrepreneur who...,"{'ORACLE CORPORATION': 'Founder', 'CERNER CORP...","{'TESLA, INC.': 'Director/Board Member', 'Rete...",{},/insider/LARRY-ELLISON-A0004C/
2,Kirk Kerkorian,NaN,NaN,NaN,NaN,{},106,"['Investment Managers', 'Consumer Services', '...","Kerkor 'Kirk' Kerkorian (born June 6, 1917), k...",{},"{'MGM RESORTS INTERNATIONAL': 'Founder', 'Trac...",{},/insider/KIRK-KERKORIAN-A00099/
3,George Soros,NaN,NaN,NaN,NaN,{},94,"['Commercial Services', 'Financial Conglomerat...",George Soros lived the Hungary’s Nazi occupati...,"{'Soros Fund Management LLC': 'Founder', 'Open...","{'Soros Economic Development Fund': 'Founder',...",{'London School of Economics & Political Scien...,/insider/GEORGE-SOROS-A000HG/
4,Bill Gates,NaN,NaN,NaN,Net worth: 42 328 M\n $ as of 2024-10-30,{'MICROSOFT CORPORATION': {'link': '/quote/sto...,68,"['Telecommunications Equipment', 'Commercial S...","William Henry ""Bill"" Gates III (born 28 Octobe...",{'Bill & Melinda Gates Foundation Trust': 'Cha...,{'MICROSOFT CORPORATION': 'Chief Executive Off...,{},/insider/BILL-GATES-A000O8/


In [19]:
result_df.head()

,Name,Age,Rank,Rank change,Wealth US$m,Change in wealth,Business,Place of Residence,Key Source of Wealth
0,Ma Yun,55,1,0,"39,000",2%,Alibaba,Zhejiang,"E-commerce, fintech"
1,Ma Huateng,48,2,1,"37,000",8%,Tencent,Guangdong,Internet services
2,Xu Jiayin,61,3,-1,"30,000",-16%,Evergrande,Guangdong,"Real estate, investments"
3,He Xiangjian,77,4,2,"26,000",38%,Midea,Guangdong,Home appliances
4,He Jianfeng,52,4,2,"26,000",38%,Midea,Guangdong,Home appliances


In [27]:
# Perform a left join on 'Name' from result_df and 'name' from insider_df
merged_df = result_df.merge(insider_df, how='left', left_on='Name', right_on='name')

In [28]:
sorted_df = merged_df.sort_values(by='name', na_position='last')
sorted_df.head()

,Name,Age,Rank,Rank change,Wealth US$m,Change in wealth,Business,Place of Residence,Key Source of Wealth,name,...,company_url,net_worth,known_holdings,age,industries,summary,active_positions,former_positions,trainings,link
2149,An Xu,56,1727,-27,290,-9%,SLAC,Jiangsu,Packaging materials,An Xu,...,NaN,91 885\n $ as of 2024-11-29,{'BANK OF GUIZHOU CO. LTD.': {'link': '/quote/...,62,"['Finance', 'Consumer Services']","Presently, An Xu holds the position of Preside...",{},"{'BANK OF GUIZHOU CO., LTD.': 'President', 'An...",{'Guizhou University of Finance & Economics': ...,/insider/AN-XU-A2QTRV/
2147,An Xu,56,1727,-27,290,-9%,SLAC,Jiangsu,Packaging materials,An Xu,...,NaN,Net worth: 89 395\n $ as of ...,{'BANK OF GUIZHOU CO. LTD.': {'link': '/quote/...,61,"['Consumer Services', 'Finance']","Presently, An Xu holds the position of Preside...",{},"{'BANK OF GUIZHOU CO., LTD.': 'President', 'An...",{'Guizhou University of Finance & Economics': ...,/insider/AN-XU-A2QTRV/
2148,An Xu,56,1727,-27,290,-9%,SLAC,Jiangsu,Packaging materials,An Xu,...,NaN,Net worth: 89 395\n $ as of ...,{'BANK OF GUIZHOU CO. LTD.': {'link': '/quote/...,61,"['Finance', 'Consumer Services']","Presently, An Xu holds the position of Preside...",{},"{'BANK OF GUIZHOU CO., LTD.': 'President', 'An...",{'Guizhou University of Finance & Economics': ...,/insider/AN-XU-A2QTRV/
585,Deng Hui,57,444,New,"1,300",new,Arcsoft,Zhejiang,IT,Deng Hui,...,/quote/stock/JD-HEALTH-INTERNATIONAL-I-119080141/,NaN,{},39,"['Miscellaneous Commercial Services', 'Commerc...",Deng Hui currently works at The Chinese Instit...,{'JD HEALTH INTERNATIONAL INC.': 'Director of ...,{},{'University of International Business & Econo...,/insider/DENG-HUI-A3RX7M/
1169,Guo Li,-,912,51,640,5%,CDH,Beijing,Investments,Guo Li,...,NaN,NaN,{},47,['Retail Trade'],Guo Li is on the board of Zhongbai Holdings Gr...,{},"{'CHENGDU HONGQI CHAIN CO.,LTD.': 'Director/Bo...",{},/insider/GUO-LI-A1UVO7/


In [33]:
sorted_df = sorted_df.dropna(subset='name')

In [34]:
sorted_df.to_csv('merged_chinese_list_names.csv', index=False)